In [1]:
import pandas as pd
import os
import numpy as np
from dataclasses import dataclass
import random
import time



np.random.seed(14)

In [2]:
@dataclass
class Particle:
    x: float
    y: float
    theta: float
    weight: float 
        


@dataclass
class LandMark:
    x: float
    y: float
    index: int

In [3]:
gt_data = pd.read_csv('data/gt_data.txt', names=['X','Y','Orientation'], sep=' ')
map_data = pd.read_csv('data/map_data.txt', names=['X','Y','# landmark'])
control_data = pd.read_csv('data/control_data.txt', names=['velocity','Yaw rate'], sep=' ')

#observation = pd.read_csv('data/observation/observations_000001.txt', names = ['X cord','Y cord'], sep=' ')


result = [(x,y, landmark) for x,y,landmark in zip(map_data['X'],map_data['Y'], map_data['# landmark'])]
landarkList=[]
for res in result:
    l = LandMark(res[0],res[1],res[2])
    landarkList.append(l)

    

a = os.listdir("data/observation")
a.sort()
observation=[]
for i in range(len(a)):
    fileName = 'data/observation/'+a[i]
    observationTmp = pd.read_csv(fileName, names = ['X cord','Y cord'], sep=' ')
    observation.append(observationTmp)

In [4]:
def calculateDistance(landmark1, landmark2):
    a =  np.sqrt((landmark1.x - landmark2.x)**2 + (landmark1.y - landmark2.y)**2)
    return a
    
 
    
def findClosestLandmark(map_landmarks, singleObs):
    
   #todo: write code:
    return closest_landmark
    
    
    
        
def getError(gt_data, bestParticle):
    error1 = np.abs(gt_data[0] - bestParticle.x)
    error2 = np.abs(gt_data[1] - bestParticle.y)
    error3 = np.abs(gt_data[2] - bestParticle.theta)
    if(error3>2*np.pi):
        error3 = 2*np.pi - error3
    return (error1, error2, error3)

def findObservationProbability(closest_landmark,map_coordinates, sigmaX, sigmaY):
    
    mew_x = closest_landmark.x
    mew_y = closest_landmark.y
    
    x = map_coordinates.x;
    y = map_coordinates.y;
    frac =  (1/ (2 * np.pi * sigmaX * sigmaY ))
    weight1 = (x-mew_x)**2/((sigmaX)**2)  +(y-mew_y)**2/(sigmaY**2)    
    ans = np.exp(-0.5*weight1)
    return ans



def mapObservationToMapCoordinates(observation, particle):
    x = observation.x
    y = observation.y

    xt = particle.x
    yt = particle.y
    theta = particle.theta

    MapX = x * np.cos(theta) - y * np.sin(theta) + xt
    MapY = x * np.sin(theta) + y * np.cos(theta) + yt

    return MapX, MapY

def mapObservationsToMapCordinatesList(observations, particle):
    
    convertedObservations=[]
    i=0
    for obs in observations.iterrows():
        singleObs = LandMark(obs[1][0],obs[1][1],1)
        mapX, mapY = mapObservationToMapCoordinates(singleObs, particle)
        tmpLandmark = LandMark(x=mapX, y=mapY, index=i)
        i+=1
        convertedObservations.append(tmpLandmark)
    return convertedObservations



In [5]:
class ParticleFilter:
    particles = []
    def __init__(self, intialX, initialY, std, numOfParticles):
        self.number_of_particles = numOfParticles
        for i in range(self.number_of_particles):
           # tmpParticle = Particle(intialX,initialY,0,1)
            x = random.gauss(intialX, std)
            y = random.gauss(initialY, std)
            theta = random.uniform(0, 2*np.pi)
            tmpParticle = Particle(x,y , theta,1)
            self.particles.append(tmpParticle)
            
            
            
    def moveParticles(self, velocity, yaw_rate, delta_t=0.1):
        
        for i in range(self.number_of_particles):
            if(yaw_rate!=0):
                theta = self.particles[i].theta
                newTheta = theta + delta_t*yaw_rate;
                newX =  self.particles[i].x +(velocity/yaw_rate)*(np.sin(newTheta)-np.sin(theta));
                newY =  self.particles[i].y +(velocity/yaw_rate)*(np.cos(theta)-np.cos(newTheta));
                
                #todo Add noise!!
                self.particles[i].x = newX +random.gauss(0, 0.3)
                self.particles[i].y = newY +random.gauss(0, 0.3)
                self.particles[i].theta = newTheta +random.gauss(0, 0.01)
            else:
                print("ZERO!!!")
            

        
        
    def UpdateWeight(self, observations):
        
        
        for i in range(self.number_of_particles):
           #todo: Write code:
            
    
    
    def getBestParticle(self):
        best_particle =  max(self.particles, key= lambda particle: particle.weight)
        return best_particle    
    
    def getBestParticleOut(self):
        x=0
        y=0
        theta=0
        for i in range(self.number_of_particles):
            x+= self.particles[i].x
            y+= self.particles[i].y
            theta+= self.particles[i].theta
        x=x/self.number_of_particles
        y=y/self.number_of_particles
        theta=theta/self.number_of_particles
        best_particle =  Particle(x,y,theta, weight=1)
        return best_particle        
    
    def PrintWeights(self):
        for i in range(self.number_of_particles):
            print("Weight:",self.particles[i].weight, self.particles[i].x,self.particles[i].y)
    
    
    def Resample(self):
        
    #todo: writecode
       
        
        

In [7]:
sigmaY = 0.3

magicNumberOfParticles = 200


start = time.time()

def main():
    particleFilter = ParticleFilter(0 ,0 ,0.3, numOfParticles=magicNumberOfParticles)
    #articleFilter = ParticleFilter(6.2785 ,1.9598 ,0.3, numOfParticles=magicNumberOfParticles)


    for i in range(len(observation)):
        #prediction
        if(i!=0):
            velocity = control_data.iloc[i-1][0]
            yaw_rate = control_data.iloc[i-1][1]
            particleFilter.moveParticles(velocity, yaw_rate)
        a = observation[i].copy()
        particleFilter.UpdateWeight(a)
        particleFilter.Resample()
        bestP = particleFilter.getBestParticle()
        error = getError(gt_data.iloc[i], bestP)
        print(i,error)
    end = time.time()
    print(end - start)
        
main()

0 (0.3384931635787778, 0.11164044648499338, 0.000804913267553847)
1 (0.05410278428460824, 0.035239268145641045, 0.004627930235767408)
2 (0.07842028347803964, 0.024184772438972768, 0.0020310191153137813)
3 (0.06893532056150597, 0.02048294740028478, 0.003385516656145615)
4 (0.046212988971949365, 0.07995071613472549, 0.0017102573487139394)
5 (0.01568067307583476, 0.09620663787845318, 0.0005155018418956936)
6 (0.0007107914430424245, 0.08677362203946659, 0.00018780232218629234)
7 (0.04333234793766039, 0.06640038091098477, 0.000283428075949943)
8 (0.06007258257727521, 0.007365263437367986, 0.0017294412726545083)
9 (0.09305132162276841, 0.0027982860205648663, 0.0012923486112322302)
10 (0.04209735401736481, 0.018332776209726287, 0.001638320749868416)
11 (0.0053590485616119565, 0.0561306870384044, 0.0012328332175792212)
12 (0.004461810817559808, 0.07324510415394725, 0.0020062037182065895)
13 (0.030729874984157135, 0.004779654686440704, 0.0010960556083822737)
14 (0.06198636412568703, 0.059882061

119 (0.04931084356718429, 0.07479554878016614, 0.002887407643760012)
120 (0.0793798551316911, 0.0184274147333916, 0.00044282602094711243)
121 (0.04353020152045417, 0.012063645576407822, 0.0006809057130585383)
122 (0.07398297836702739, 0.05980046264393124, 0.0011015870489125418)
123 (0.019352885545984577, 0.053498114212278125, 0.002276978618549322)
124 (0.05430349548082347, 0.02367174221805257, 0.0008275439906188925)
125 (0.04866547459593562, 0.016536776511966167, 0.0021480944454292106)
126 (0.04614422650367089, 0.03013595602150687, 0.0023000396512654203)
127 (0.005081384218016183, 0.035954206549494216, 0.0025910299098575174)
128 (0.028619130864058206, 0.022522074406651882, 0.0008591790362495466)
129 (0.07715678781022461, 0.04245404205333969, 0.0030080676890263253)
130 (0.03119497680727079, 0.007041437298980213, 0.002183450481978544)
131 (0.028693702286901157, 0.0014156308109782856, 0.0007500978646807488)
132 (0.020364126577106845, 0.10182751458718187, 0.0002097695635661645)
133 (0.0091

235 (0.009214923527224528, 0.045377197793957436, 0.0007827444394751075)
236 (0.0695800921936609, 0.018687304831249918, 0.0008918355733578409)
237 (0.15260539895851366, 0.008137745188101064, 0.004056528200449744)
238 (0.14172662238632938, 0.08048608610296526, 0.0024225212113169192)
239 (0.05134874982644533, 0.1236317710881778, 0.00013842066953007048)
240 (0.00991697959358362, 0.0020957500583946853, 0.00031943559127434185)
241 (0.017347801334835822, 0.03755802957432053, 0.001964614094317696)
242 (0.06526776545234725, 0.004779938718726839, 0.0036828048345496356)
243 (0.03834139855973717, 0.010336028408989506, 0.001971993074049827)
244 (0.11566468678927322, 0.013607193995017752, 0.001883378194966312)
245 (0.015916170526452333, 0.05429530163373286, 0.0021480384398637753)
246 (0.03401553100769661, 0.06455517981829928, 0.0034835538813222655)
247 (0.0560802224198369, 0.013167351564867147, 0.0006929876752543862)
248 (0.08569788726998695, 0.016321595643745468, 0.0013734885190839208)
249 (0.05431

352 (0.09979211357997997, 0.11257348697488467, 0.0007439942759104312)
353 (0.012702817788834864, 0.03294037008121009, 0.0016576877778984311)
354 (0.05163033043962173, 0.11414812124653295, 0.0014446263623422162)
355 (0.06542447007927876, 0.06167832614459101, 0.0006391308218152147)
356 (0.00810997282528092, 0.06454001231975326, 0.0004663156039477201)
357 (0.07542840008133567, 0.05037490109631193, 0.002289831108054141)
358 (0.01729078695473163, 0.052596046619633796, 0.0008421433604599571)
359 (0.04874135831286708, 0.03952441418999442, 0.0005526043966037619)
360 (0.12676629519683047, 0.01178082609718345, 0.0029322412522736485)
361 (0.018156347317358268, 0.03986240631821758, 0.0028092197561258025)
362 (0.05627508708789719, 0.03584412074123655, 9.162614678093206e-06)
363 (0.05743255729115049, 0.06367060318048168, 0.0009869391838912556)
364 (0.03816257909949172, 0.06148394659049039, 0.0031272636701480394)
365 (0.11049479785296512, 0.024995276822224355, 0.003989228706351877)
366 (0.14447222545

468 (0.059875725657590806, 0.010761700979696087, 0.000564731845244193)
469 (0.031667411997958084, 0.11606402602602017, 0.00012424784878595219)
470 (0.03502774451807511, 0.08674394111975303, 0.0008039374350699013)
471 (0.006393128847577145, 0.10602980353612423, 2.9434062261302074e-05)
472 (0.06978973524292087, 0.012927235557210537, 0.0016079830689772479)
473 (0.03810688088333336, 0.03994280726389121, 0.00147833025295796)
474 (0.027644615672627992, 0.056981557076994704, 0.0015549072396874308)
475 (0.034865030383684825, 0.025452608537232635, 0.0030092364757621226)
476 (0.07098611276262545, 0.011448213982355071, 0.00271688548226523)
477 (0.10017081951848184, 0.020017269535472337, 0.0004519701817087274)
478 (0.040477800869908265, 0.04192490524837922, 0.0009782370185713596)
479 (0.06458734690424706, 0.04373617357054371, 0.0004363767416806219)
480 (0.002179917462655112, 0.029095047685629538, 0.0001297951378664841)
481 (0.02777139271300655, 0.03060904332207315, 9.547228327511093e-05)
482 (0.00

...

1971 (0.0679813004272205, 0.04465712272147471, 0.0016595058767414628)
1972 (0.10445056426550536, 0.011203064130526741, 0.0014253770020706824)
1973 (0.1174413551460134, 0.05368089358355288, 0.005209579678877141)
1974 (0.038219356240347224, 0.01524776017880658, 6.576217303866372e-05)
1975 (0.037630073562752386, 0.05645848285238131, 0.0005132718048548313)
1976 (0.1465070545871754, 0.0036536451436042228, 0.005324651553981319)
1977 (0.08339578723672503, 0.01147874540040128, 0.003909942248058407)
1978 (0.020325559943955795, 0.09144252881143977, 0.000662124651518603)
1979 (0.01714101244289168, 0.03856760058936892, 0.0013511607250920044)
1980 (0.049753493771959256, 0.03154681421789185, 0.0004587546481240601)
1981 (0.1112753137080631, 0.007840888087848441, 0.0017913131849471853)
1982 (0.030786437578058212, 0.06499266218134458, 0.00020329345307557034)
1983 (0.0698922758392655, 0.016145567222533685, 0.0013889316655113504)
1984 (0.145253194587724, 0.031472516277437634, 0.0037529475440871707)
1985 

2086 (0.04241387124454832, 0.041144251459279246, 0.001161754763547851)
2087 (0.047696639322879264, 0.004698233937588725, 0.0020291323927730254)
2088 (0.11749055102379202, 0.06197922430275149, 0.0025505526896942854)
2089 (0.14102368910891983, 0.07292986777894761, 0.003765162728921023)
2090 (0.03530242685792473, 0.09689797135217937, 0.0010233964766155346)
2091 (0.023745096057865567, 0.03868146236822234, 0.00029580274909912774)
2092 (0.027997616098559774, 0.01428548077236158, 0.0012988149900734491)
2093 (0.06926514043593102, 0.04059777566194889, 0.0023904510832108983)
2094 (0.02194552690286855, 0.07753335220262159, 0.001234308151131458)
2095 (0.07453652711663494, 0.016682569647640477, 0.00044416788486323355)
2096 (0.123005413242538, 0.012648751229551891, 0.0013037480296049786)
2097 (0.02285032263111475, 0.007952356849941111, 0.0009230923660039636)
2098 (0.04491265160618241, 0.04966252836042351, 0.0013645782268869588)
2099 (0.0568768246880893, 0.08816215659891746, 0.0006503928055865416)
21

2201 (0.080461421282493, 0.009041547781777126, 0.0004678067402208441)
2202 (0.00892290759840364, 0.0408435603729842, 0.00010229125008054396)
2203 (0.001264905909152958, 0.04408752694460105, 0.00015411143587895637)
2204 (0.009937424811582218, 0.011830446588056986, 0.0008736078409348202)
2205 (0.03010434264345463, 0.07518171240187854, 0.003162538887336641)
2206 (0.025075066426268222, 0.024896270291300482, 0.00011276731419007646)
2207 (0.006493710782748963, 0.05345641532206713, 0.00017477061756165924)
2208 (0.0033852384697468096, 0.05990985450138453, 0.0015545832299443951)
2209 (0.04106430795138749, 0.12778528736036066, 0.0017255664717259478)
2210 (0.02804844491561198, 0.013316302195981677, 0.0025235859945360595)
2211 (0.051775068406755054, 0.03379918675338445, 0.0003269860579950823)
2212 (0.00340796542843691, 0.0099816243138946, 0.001319393602856156)
2213 (0.010461088757750758, 0.1172245273533008, 0.0010723859307364414)
2214 (0.026933843899016097, 0.03223969341242139, 0.00071624117837032

2316 (0.004420165677252896, 0.028411961837491617, 0.00014181312454741857)
2317 (0.049483978093407543, 0.016266153247144644, 0.0015446368506468744)
2318 (0.028376246992657173, 0.04500407695067565, 0.0005061143407032453)
2319 (0.11572906402742689, 0.021172688376452697, 0.0011138943722506234)
2320 (0.07793284278765178, 0.0350232603312719, 0.0019335963269804246)
2321 (0.006975219665582344, 0.0018237725557810336, 0.0025591374982338877)
2322 (0.041451100796075835, 0.019163710099604714, 0.0006105235397386721)
2323 (0.01969782387153174, 0.05140570782897669, 0.0015254734904650569)
2324 (0.07001985920947362, 2.732520611914424e-05, 0.00035050533940550466)
2325 (0.09222606373996456, 0.010698698369438375, 0.0010162224083853921)
2326 (0.0037710661368510756, 0.025419431698836092, 0.0017318912877262171)
2327 (0.025702928884646425, 0.08937217783730489, 0.0013713188809143517)
2328 (0.0458599666749846, 0.02322626526617455, 0.0016495779165799362)
2329 (0.07364399762955998, 0.12671132184763678, 0.002955223

2431 (0.08788619962815147, 0.08306385136627625, 0.002906927248956581)
2432 (0.07579504058103481, 0.038607781084451176, 0.0021564110559781913)
2433 (0.024405527111539982, 0.04755876224836442, 0.00022701221894272372)
2434 (0.08198977773590599, 0.06921086243590224, 0.0004917127845158742)
2435 (0.03475789426081377, 0.05120894296591416, 0.0005960395491739767)
2436 (0.054934453723298304, 0.03370797258407521, 4.386609112239093e-05)
2437 (0.028021970199560542, 0.06296024137535738, 0.0029669949951964814)
2438 (0.04524714573010158, 0.014856485683949927, 0.0010941652253202427)
2439 (0.04198532439306035, 0.03768143150750092, 0.0031877392147408035)
2440 (0.0358546290250672, 0.02736926085982816, 0.00039684589188726527)
2441 (0.027086038866833917, 0.02157002287712828, 0.0006078326250078536)
2442 (0.022283732218078, 0.02491766933236761, 0.000563369547442516)
2443 (0.1875303148527223, 0.06416427564953153, 0.0030193337093322725)


In [ ]:
import cv2

In [ ]:
# WIDTH=1600
# HEIGHT=1200

# def mouseCallback(event, x, y, flags,null):
#     global i
    

# WINDOW_NAME="Particle Filter"
# img = np.zeros((HEIGHT,WIDTH,3), np.uint8)
# cv2.namedWindow(WINDOW_NAME)
# DELAY_MSEC=50
# #cv2.setMouseCallback(WINDOW_NAME,mouseCallback)

# i=1


    

# particleFilter = ParticleFilter(6.2785 ,1.9598 ,0.3, numOfParticles=50)

# while(1):
 
#     cv2.imshow(WINDOW_NAME,img)
#     img = np.zeros((HEIGHT,WIDTH,3), np.uint8)
    
#     if(i>2000):
#         break
    
#     for landmark in landarkList:
#         cv2.circle(img,tuple((int(landmark.x)*2+400, int(landmark.y)*2+300)),3,(255,0,0),-1)
    
#     #draw_particles:
#     for particle in particleFilter.particles:
#         cv2.circle(img,tuple((int(particle.x)*2+400,int(particle.y)*2+300)),1,(255,255,255),-1)
#     cv2.circle(img,tuple((int(gt_data.iloc[i].X)*2+400,int(gt_data.iloc[i].Y)*2+300)),1,(0,255,0),-1)
    
#     if cv2.waitKey(DELAY_MSEC) & 0xFF == 27:
#         break
#     if cv2.waitKey(33) == ord('a'):
#         velocity = control_data.iloc[i-1][0]
#         yaw_rate = control_data.iloc[i-1][1]
#         particleFilter.moveParticles(velocity, yaw_rate)
#         a = observation[i].copy()
#         particleFilter.UpdateWeight(a)
#         particleFilter.PrintWeights()
#         particleFilter.Resample()
        
#         i+=1

# cv2.destroyAllWindows()
